<a href="https://colab.research.google.com/github/Meridor6919/BeeClassification/blob/Functional-API/BeeClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras import layers
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

In [2]:
dataset = tfds.load('bee_dataset', split=['train'], as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/7490 [00:00<?, ? examples/s]

Shuffling ~/tensorflow_datasets/bee_dataset/bee_dataset_300/1.0.0.incompleteMQY6GT/bee_dataset-train.tfrecord*…

Dataset bee_dataset downloaded and prepared to ~/tensorflow_datasets/bee_dataset/bee_dataset_300/1.0.0. Subsequent calls will reuse this data.


In [8]:
# Split between training data, validation data, test data
splitSize = (0.0, 0.765, 0.9, 1.0)

array = np.vstack(list(tfds.as_numpy(dataset[0])))

ranges = list([int(array.shape[0] * x) for x in splitSize])

def getDataset(ranges, idx):
    return (
        np.array(list(map(lambda x: x[0][:, :, 0], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['cooling_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['pollen_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['varroa_output'], array[ranges[idx]: ranges[idx + 1]]))),
        np.array(list(map(lambda x: x[1]['wasps_output'], array[ranges[idx]: ranges[idx + 1]])))
    )

# 0 - train, 1 - validation, 2 - test
   
_input = keras.Input(shape=(300, 150, 1), name="img")
x = layers.Conv2D(256, (3, 3), activation='relu')(_input)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='tanh')(x)
x = layers.MaxPooling2D((2, 2))(x)
x = layers.Conv2D(128, (3, 3), activation='tanh')(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='tanh')(x)
_output = layers.Dense(4)(x)


# wszystkie dane na raz (z 4 parametrow robimy wektory czteroelementowe)
labels_all = [None] * 3
images = [None] *3 
for i in range(3):
    helper = getDataset(ranges, i)
    images[i] = helper[0]
    labels_all[i] = np.column_stack((helper[1], helper[2], helper[3], helper[4]))

#print(str(labels_all[0].shape))
#print(str(labels_all[1].shape))

model_all = keras.Model(_input, _output, name="model_all")
# model_all.compile(optimizer=tf.keras.optimizers.Adadelta(), loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
# tf.keras.losses.CategoricalCrossentropy()
model_all.compile(optimizer=tf.keras.optimizers.Adadelta(), loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history_all = model_all.fit(images[0], labels_all[0], validation_data=(images[1], labels_all[1]), epochs=10)


print(max(history_all.history['accuracy']))

Epoch 1/10
180/180 [==============================] - 44s 231ms/step - loss: 0.7944 - accuracy: 0.3409 - val_loss: 0.8502 - val_accuracy: 0.6215
Epoch 2/10
180/180 [==============================] - 41s 227ms/step - loss: 0.7254 - accuracy: 0.3224 - val_loss: 0.7982 - val_accuracy: 0.6532
Epoch 3/10
180/180 [==============================] - 41s 226ms/step - loss: 0.6657 - accuracy: 0.3802 - val_loss: 0.6775 - val_accuracy: 0.4111
Epoch 4/10
180/180 [==============================] - 41s 226ms/step - loss: 0.6202 - accuracy: 0.4292 - val_loss: 0.7305 - val_accuracy: 0.1858
Epoch 5/10
180/180 [==============================] - 41s 226ms/step - loss: 0.5905 - accuracy: 0.4559 - val_loss: 0.6436 - val_accuracy: 0.3014
Epoch 6/10
180/180 [==============================] - 43s 240ms/step - loss: 0.5650 - accuracy: 0.4779 - val_loss: 0.6286 - val_accuracy: 0.4121
Epoch 7/10
180/180 [==============================] - 41s 226ms/step - loss: 0.5480 - accuracy: 0.4987 - val_loss: 0.5373 - val_ac